[Authenticating to sovereign cloud](https://learn.microsoft.com/en-us/python/api/overview/azure/search-documents-readme?view=azure-python#authenticate-in-a-national-cloud)

In [ ]:
from pathlib import Path
import os
import json
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, AzureAuthorityHosts
from azure.search.documents import SearchClient

env_directory = Path.cwd() / ".azure"

env_file = next(
    (candidate / ".env" for candidate in env_directory.glob("avcoe-*")),
    None,
)

if env_file is None:
    raise FileNotFoundError(
        "Could not locate an avcoe-* environment directory under .azure"
    )

load_dotenv(dotenv_path=env_file, override=False)

search_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]

search_index = os.environ["SEARCH_INDEX_NAME"]

print(f"Using search endpoint {search_endpoint}, and index: {search_index}")

authority_host = (
    AzureAuthorityHosts.AZURE_GOVERNMENT
    if os.getenv("CLOUD_NAME") == "AzureUSGovernment"
    else AzureAuthorityHosts.AZURE_PUBLIC_CLOUD
)

print(f"Using authority host: {authority_host}")

In [ ]:
# Set the correct credential scope for Azure Government
audience = ("https://search.azure.us" if os.getenv("CLOUD_NAME") == "AzureUSGovernment"else "https://search.azure.com")

print(f"Using credential scope: {audience}")

credential = DefaultAzureCredential()

# Create SearchClient with the scoped credential
search_client = SearchClient(search_endpoint, search_index, credential, audience=audience)

In [ ]:
results = search_client.search("*",
                                 facets=["title"],
)

results.get_facets()

In [ ]:
import pandas as pd

# Get the facet results
facet_results = results.get_facets()['title']

# Convert to DataFrame
df = pd.DataFrame(facet_results)

# Display as a table
df

In [ ]:
doc_list = list(df.value)

In [ ]:

results = search_client.search("what is an example of a group 2 UAS",
                                 top=3,
                                 include_total_count=True,
                                 query_type="semantic",
                                 filter=f"title eq '{doc_list[3]}'",
                                 select=["title", "chunk"],)
                                 
for result in results:
    print(json.dumps(dict(result), indent=2, default=str))